## 1. Sentimentanalyse mit VADER

VADER (Valence Aware Dictionary and sEntiment Reasoner) ist ein Tool für Sentimentanalyse, das besonders auf Texte aus den sozialen Medien verwendet wird. Die Sentimentanalyse basiert auf einem manuell annotierten englischen Lexikon (inkl. Emojis, Abkürzungen, Umgangssprache). ([Quelle](https://github.com/cjhutto/vaderSentiment)) 

Hier testen wir VADER auf [deutsche Kaffeebewertungen](https://www.kaggle.com/mldado/german-online-reviewsratings-of-organic-coffee). 

In [1]:
import pandas as pd   #für Datenanalyse

import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download("vader_lexicon")
sia = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/lorena/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [2]:
bewertungen = pd.read_csv("kaffee_reviews.csv")
bewertungen.tail() # zeigt die letzten 5 Zeilen

,Unnamed: 0,brand,rating,review
459,459,GEPA Kaffee,5,"Ich bevorzuge Gepa Kaffee, da er FairTrade pro..."
460,460,GEPA Kaffee,5,Kaffee genießen mit gutem Gefühl: besonders zu...
461,461,GEPA Kaffee,5,"Ich habe einen magenfreundlichen, aber koffein..."
462,462,GEPA Kaffee,5,Bei Gepa unterstützt man bei jedem Kauf ein we...
463,463,GEPA Kaffee,5,Schon lange gehört Kaffee von Gepa zum festen ...


In [3]:
# die Spalten "Unnamed" und "brand" entfernen, weil wir sie für unsere Analyse nicht brauchen

kaffee_bewertungen = bewertungen.drop(["Unnamed: 0", "brand"], axis=1)
kaffee_bewertungen.head() # zeigt die ersten 5 Zeilen

,rating,review
0,5,Wenn ich Bohnenkaffee trinke (auf Arbeit trink...
1,5,Für mich ist dieser Kaffee ideal. Die Grundvor...
2,5,Ich persönlich bin insbesondere von dem Geschm...
3,5,ganz abgesehen vom geschmack legt gepa inzwisc...
4,5,Seit Jahren kaufe ich am liebsten den Kaffee u...


In [4]:
# die Anzahl der Bewertungen im jeweiligen Rating anzeigen

kaffee_bewertungen.rating.value_counts()

5    346
4     87
1     23
3      8
Name: rating, dtype: int64

Aus den 464 Bewertungen im Dataset, 433 sind positiv (Ratings 4 und 5), 23 sind sehr negativ (Rating 1) und 8 sind neutral (Rating 3).

In [5]:
# lambda x:

kaffee_bewertungen["neg"] = kaffee_bewertungen["review"].apply(lambda x:sia.polarity_scores(x)["neg"])
kaffee_bewertungen["pos"] = kaffee_bewertungen["review"].apply(lambda x:sia.polarity_scores(x)["pos"])
kaffee_bewertungen["neu"] = kaffee_bewertungen["review"].apply(lambda x:sia.polarity_scores(x)["neu"])
kaffee_bewertungen["compound"] = kaffee_bewertungen["review"].apply(lambda x:sia.polarity_scores(x)["compound"])

In [6]:
# die Ergebnisse darstellen und in aufsteigender Reihenfolge nach Rating sortieren,
# damit wir die Sentiment Werte mit den extremen Ratings (1 und 5) vergleichen können.

kaffee_bewertungen.sort_values(by="rating", ascending=True)

,rating,review,neg,pos,neu,compound
38,1,"Naturland gibt auf ihrer Internetseite an, das...",0.227,0.000,0.773,-0.9485
36,1,Ich nutze von REWE Bio Röstkaffee Fairtrade di...,0.139,0.106,0.755,-0.4389
37,1,Naturland von Rewe,0.000,0.000,1.000,0.0000
75,1,…denn auf der Website wird ausführlich erläute...,0.135,0.000,0.865,-0.9485
102,1,Aus einigen Testberichten verschiedener Intern...,0.206,0.000,0.794,-0.9136
...,...,...,...,...,...,...
165,5,Wir haben in der Vergangenheit den SCHIRMER bi...,0.000,0.032,0.968,0.3182
164,5,"Bio drauf und bio drin, genau so wie es sein s...",0.000,0.000,1.000,0.0000
163,5,"das ist in sehr Geschmackvoller Kaffee,den ich...",0.000,0.078,0.922,0.3182
137,5,Kundenbetreuung: 5 Sterne,0.000,0.000,1.000,0.0000


Die Werte **'neg'**, **'pos'** und **'neu'** geben an, wie negativ, positiv bzw. neutral eine Bewertung ist. Die Werte liegen zwischen 0 und 1 und die Summe der drei Werte soll 1 (100%) ergeben. Der Wert **'compound'** ergibt sich aus der Summe aller Bewertungen eines Wortes aus dem VADER Lexikon, die zwischen -1 (negativ) und +1 (positiv) normalisiert werden. ([mehr Infos hier](https://github.com/cjhutto/vaderSentiment#about-the-scoring))

Aus den 10 angezeigten Bewertungen ergeben sich deutliche Unterschiede zwischen den sehr positiven (Rating 5) und sehr negativen (Rating 1) Bewertungen.
- Die'compound'-Werte sind negativ für die negativen Bewertungen bzw. positiv für die positive Bewertungen.
- Die 'neg'-Werte liegen bei ~ 20% für die negativen Bewertungen bzw. 0% für die positiven Bewertungen.
- Die 'pos'-Werte sind höher für die positiven Bewertungen als für die negativen.
- Die 'neu'-Werte sind höher für die positiven Bewertungen als für die negativen, aber der Unterschied ist nicht groß.

Im Allgemeinen stimmen die Werte der Sentimenanalyse mit den Ratings der Bewertungen.

## 2. Sentimentanalyse mit TextBlob



In [7]:
from textblob_de import TextBlobDE as TextBlob

In [8]:
kaffee_bewertungen["polarity"] = kaffee_bewertungen.apply(lambda x: TextBlob(x["review"]).sentiment.polarity, axis=1)
kaffee_bewertungen["subjectivity"] = kaffee_bewertungen.apply(lambda x: TextBlob(x["review"]).sentiment.subjectivity, axis=1)

In [9]:
kaffee_bewertungen.tail()

,rating,review,neg,pos,neu,compound,polarity,subjectivity
459,5,"Ich bevorzuge Gepa Kaffee, da er FairTrade pro...",0.089,0.000,0.911,-0.6688,0.333333,0.166667
460,5,Kaffee genießen mit gutem Gefühl: besonders zu...,0.000,0.000,1.000,0.0000,0.150000,0.000000
461,5,"Ich habe einen magenfreundlichen, aber koffein...",0.000,0.000,1.000,0.0000,0.850000,0.000000
462,5,Bei Gepa unterstützt man bei jedem Kauf ein we...,0.000,0.000,1.000,0.0000,0.850000,0.000000
463,5,Schon lange gehört Kaffee von Gepa zum festen ...,0.000,0.126,0.874,0.7351,0.400000,0.000000


In [10]:
kaffee_bewertungen.sort_values(by="rating", ascending=True)

,rating,review,neg,pos,neu,compound,polarity,subjectivity
38,1,"Naturland gibt auf ihrer Internetseite an, das...",0.227,0.000,0.773,-0.9485,0.470000,0.000000
36,1,Ich nutze von REWE Bio Röstkaffee Fairtrade di...,0.139,0.106,0.755,-0.4389,0.000000,0.000000
37,1,Naturland von Rewe,0.000,0.000,1.000,0.0000,0.000000,0.000000
75,1,…denn auf der Website wird ausführlich erläute...,0.135,0.000,0.865,-0.9485,0.000000,0.142857
102,1,Aus einigen Testberichten verschiedener Intern...,0.206,0.000,0.794,-0.9136,-0.550000,0.000000
...,...,...,...,...,...,...,...,...
165,5,Wir haben in der Vergangenheit den SCHIRMER bi...,0.000,0.032,0.968,0.3182,0.340000,0.000000
164,5,"Bio drauf und bio drin, genau so wie es sein s...",0.000,0.000,1.000,0.0000,0.166667,0.000000
163,5,"das ist in sehr Geschmackvoller Kaffee,den ich...",0.000,0.078,0.922,0.3182,0.075000,0.000000
137,5,Kundenbetreuung: 5 Sterne,0.000,0.000,1.000,0.0000,0.000000,0.000000


- Der Wert **'polarity'**  liegt zwischen [-1,1] und gibt an wie negativ/positiv eine Bewertung ist.
- Der Wert **'subjectivity'** liegt zwischen [0,1] und gibt an wie objektiv/subjektiv eine Bewertung ist.

Aus den 10 angezeigten Bewertungen geht hervor, dass die 'polarity'-Werte der negativen Bewertungen negativ bzw. 0, während die der positiven Bewertungen positiv sind. Die 'subjectivity'-Werte sind 0 sowohl für die positive als auch für negative Bewertungen, was bedeutet, dass die Bewertungen objektiv sind (oder dass das Modell nicht gut funktionert). Im Allgemeinen stimmen die Werte der Sentimenanalyse mit den Ratings der Bewertungen.

## 3. SpaCy + SentiWS

In [11]:
# zuerst: pip install spacy-sentiws
import spacy
from spacy_sentiws import spaCySentiWS

"Der SentimentWortschatz, oder kurz SentiWS, ist eine öffentlich verfügbare deutschsprachige Ressource für die Sentiment Analyse, Opinion Mining und ähnliche Zwecke. Dabei werden für enthaltene Wörter die positive und negative Polarität im Intervall [-1; 1] angegeben, sowie deren Wortart und (falls anwendbar) Flexionsvarianten. Die aktuelle Version des SentiWS enthält ungefähr 1.650 positive und 1.800 negative Grundformen, so dass, inklusive der verschiedenen Flexionsformen, insgesamt etwa 16.000 positive und 18.000 negative Wortformen enthalten sind. SentiWS enthält nicht nur Adjektive und Adverbien, sondern auch Nomen und Verben die Träger von Sentiment sind." ([Quelle](https://wortschatz.uni-leipzig.de/de/download))

In [12]:
from spacy.tokens import Token
from spacy_sentiws.senti_ws_wrapper import SentiWSWrapper


class spaCySentiWS(object):
    def __init__(self, sentiws_path):
        self.sentiws = SentiWSWrapper(sentiws_path=sentiws_path)
        Token.set_extension("sentiws", getter=self.get_sentiment, force=True)

    def __call__(self, doc):
        for token in doc:
            token._.sentiws = self.get_sentiment(token)
        return doc

    def get_sentiment(self, token):
        return self.sentiws.determine(token.text, pos_universal_google=token.pos_)

In [13]:
# einer der drei SpaCy Modelle für Deutsch laden. 'md' = medium
nlp = spacy.load("de_core_news_md")

sentiws = spaCySentiWS(sentiws_path="./spacy_sentiws/SentiWS_v2.0")
nlp.add_pipe(sentiws)

# auf einen Beispielsatz testen
doc = nlp("Mir schmeckt dieser Kaffee super. Ich trinke ihn bewusst und mit Genuss.")

# jedes Wort im Beispielsatz tokenisieren und ihm das Wert aus dem Wortschatz zuweisen
for token in doc:
    print("{}, {}, {}".format(token.text, token._.sentiws, token.pos_))

Mir, None, PRON
schmeckt, None, VERB
dieser, None, DET
Kaffee, None, NOUN
super, 0.5012, ADJ
., None, PUNCT
Ich, None, PRON
trinke, None, VERB
ihn, None, PRON
bewusst, None, ADJ
und, None, CCONJ
mit, None, ADP
Genuss, 0.0701, NOUN
., None, PUNCT


Zwei "Sentiment"-Wörter wurden bewertet: *super* als positiv (0.5) und *Genuss* als neutral (0.07). Der Satz ist aber sehr positiv.

## 3. SpaCy 

In [12]:
import spacy
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline

In [10]:
uni_bewertungen = pd.read_csv("studycheckv2.csv")
uni_bewertungen = uni_bewertungen.drop(["id", "url"], axis=1)
uni_bewertungen.tail()

,inhalt,weiter_empfehlung
221716,"Liebe Studenten und Studieninteressierte, an ...",True
221717,Studiert es besser nicht! Nach 2 Semestern war...,False
221718,Habe den besagten Studiengang zwei Semester in...,False
221719,"Ja, ja, die liebe Hochschule Rhein Waal. Wenn ...",False
221720,"Es ist gut zu sehen, dass man das soziale mit ...",True


In [11]:
# die Weiterempfehlungswerte numerisch enkodieren

uni_bewertungen["weiter_empfehlung"] = uni_bewertungen["weiter_empfehlung"].astype(int)
uni_bewertungen.tail()

,inhalt,weiter_empfehlung
221716,"Liebe Studenten und Studieninteressierte, an ...",1
221717,Studiert es besser nicht! Nach 2 Semestern war...,0
221718,Habe den besagten Studiengang zwei Semester in...,0
221719,"Ja, ja, die liebe Hochschule Rhein Waal. Wenn ...",0
221720,"Es ist gut zu sehen, dass man das soziale mit ...",1


In [12]:
uni_bewertungen["weiter_empfehlung"].value_counts()

1    199287
0     22434
Name: weiter_empfehlung, dtype: int64

In diesem Dataset gibt es 199287 positive und 22434 negative Bewertungen.

In [9]:
# Stoppwörter entfernen 

import string
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.de import German

In [13]:
nlp = spacy.load("de_core_news_md")

In [11]:
from spacy.lang.de import German
parser = German()

### Tokenisieren

In [13]:
# erste Bewertung tokenisieren (TEST)
nltk.word_tokenize(uni_bewertungen["inhalt"][0])

['Die',
 'Ausrüstung',
 'in',
 'den',
 'Laboren',
 'ist',
 'sehr',
 'modern',
 'und',
 'neu',
 'sowie',
 'technisch',
 'hochwertig',
 '.',
 'Die',
 'Anleitungen',
 'im',
 'Labor',
 'sind',
 'ebenfalls',
 'gut',
 '(',
 'u.a',
 '.',
 'da',
 'man',
 'sich',
 'auch',
 'auf',
 'die',
 'Kurse',
 'vorbereiten',
 'muss',
 ')',
 '.',
 'Zwar',
 'sind',
 'alle',
 'Vorlesungen',
 'auf',
 'englisch',
 ',',
 'jedoch',
 'kann',
 'man',
 'sich',
 'schnell',
 'reinhören',
 '.',
 'Doch',
 'wenn',
 'man',
 'Fragen',
 'hat',
 'so',
 'stehen',
 'die',
 'Professoren',
 'nach',
 'den',
 'Vorlesungen',
 'auch',
 'gerne',
 'auf',
 'deutsch',
 'mit',
 'Rat',
 'zurseite',
 '.']

### Stemming

In [14]:
from nltk.stem import SnowballStemmer
stemmer = SnowballStemmer("german")

inhalt_df = uni_bewertungen.inhalt
inhalt_corpus = inhalt_df.inhalt.str.cat(sep = '\n')
inhalt_corpus[0:1000]
inhalt_text = nltk.Text(inhalt_df)
#positive_tokens = [stemmer.stem(x) for x in positive_tokens]

AttributeError: 'Series' object has no attribute 'inhalt'

In [2]:
from nltk.corpus import stopwords

stop_words = set(stopwords.words('german'))

In [3]:
# csv Datei in dataframe konvertieren
uni_bewertungen_df = pd.read_csv("studycheckv2.csv")

uni_bewertungen_df['inhalt_tokenized'] = uni_bewertungen_df.apply(lambda row: nltk.word_tokenize(row['inhalt']), axis=1)
uni_bewertungen_df['inhalt_laenge'] = uni_bewertungen_df.apply(lambda row: len(row['inhalt_tokenized']), axis=1)
uni_bewertungen_df.head()

,id,url,inhalt,weiter_empfehlung,inhalt_tokenized,inhalt_laenge
0,41169,https://www.studycheck.de/studium/bioingenieur...,Die Ausrüstung in den Laboren ist sehr modern ...,True,"[Die, Ausrüstung, in, den, Laboren, ist, sehr,...",69
1,41171,https://www.studycheck.de/studium/internationa...,Ein Studium in International Business an der H...,True,"[Ein, Studium, in, International, Business, an...",72
2,41173,https://www.studycheck.de/studium/internationa...,Allen voran: Der Campus - zumindest die baulic...,False,"[Allen, voran, :, Der, Campus, -, zumindest, d...",434
3,41175,https://www.studycheck.de/studium/medieninform...,Das schlechte zuerst: das Campusleben wird dad...,True,"[Das, schlechte, zuerst, :, das, Campusleben, ...",601
4,41177,https://www.studycheck.de/studium/arbeits-und-...,"Da es sich um eine neuere HS handelt, herrsche...",True,"[Da, es, sich, um, eine, neuere, HS, handelt, ...",159


In [4]:
uni_bewertungen_df["weiter_empfehlung"] = uni_bewertungen_df["weiter_empfehlung"].astype(int)
uni_bewertungen_df = uni_bewertungen_df.drop(["id", "url"], axis=1)
uni_bewertungen_df.head()

,inhalt,weiter_empfehlung,inhalt_tokenized,inhalt_laenge
0,Die Ausrüstung in den Laboren ist sehr modern ...,1,"[Die, Ausrüstung, in, den, Laboren, ist, sehr,...",69
1,Ein Studium in International Business an der H...,1,"[Ein, Studium, in, International, Business, an...",72
2,Allen voran: Der Campus - zumindest die baulic...,0,"[Allen, voran, :, Der, Campus, -, zumindest, d...",434
3,Das schlechte zuerst: das Campusleben wird dad...,1,"[Das, schlechte, zuerst, :, das, Campusleben, ...",601
4,"Da es sich um eine neuere HS handelt, herrsche...",1,"[Da, es, sich, um, eine, neuere, HS, handelt, ...",159


In [52]:
import numpy as np
np.savetxt(r'studycheckv2.txt', uni_bewertungen_df.weiter_empfehlung, fmt='%d', delimiter="\t", header="inhalt\tweiter_empfehlung\tinhalt_tokenized\tinhalt_laenge")

In [ ]:
tfile = open('unibewertungen.txt', 'a')
tfile.write(uni_bewertungen_df.to_string())
tfile.close()

In [46]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer, TfidfTransformer
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

### Sonderzeichen/Stoppwörter entfernen

In [6]:
import string
punct = string.punctuation
punct

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [62]:
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer

german_stop_words = stopwords.words('german')

vect = CountVectorizer(stop_words = german_stop_words) # Now use this in your pipeline

In [36]:
def stop_word_removal(x):
    token = x.split()
    return ' '.join([w for w in token if not w in german_stop_words])


uni_bewertungen_df['removed_stop_word']  = uni_bewertungen_df['inhalt'].apply(stop_word_removal)

In [38]:
stop_word_removal("Hallo, wie geht's dir?")

"Hallo, geht's dir?"

In [58]:
from sklearn.svm import LinearSVC

In [59]:
tfidf = TfidfVectorizer(tokenizer = vect)
classifier = LinearSVC()

### Train-Test split

In [39]:
X = uni_bewertungen_df["inhalt"]
y = uni_bewertungen_df["weiter_empfehlung"]

In [40]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [41]:
X_train.shape, X_test.shape

((177376,), (44345,))

In [42]:
X_train

6168      Da\n die Hdpk eine private Hochschule ist, kos...
112126    Das Studium an der ISM bringt Spaß, die LehrIn...
159706    Meine Studium würde ich mit einer Weltreise gl...
152989    Ich bin schon Erzieherin und bringe daher scho...
88063     Ich habe gelernt mich kritisch mit vielen Vers...
                                ...                        
119879    Die Erfahrungen waren sehr unterschiedlich. Es...
103694    Ideenwelten werden real.\r\nViel mehr kann ich...
131932    Ich finde, dass man zwar in kurzer Zeit viel S...
146867    Ich finde besonders super, dass die Uni durch ...
121958    Ich habe meinen Bachelor an einer anderen Uni ...
Name: inhalt, Length: 177376, dtype: object

In [60]:
clf = Pipeline([("tfidf", tfidf), ("clf", classifier)])

In [63]:
clf.fit(X_train, y_train)

TypeError: 'CountVectorizer' object is not callable

In [65]:
pipeline = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
])

NameError: name 'stopWords' is not defined

In [54]:
clf.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('tfidf',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True))],
         verbose=False)

In [67]:
y_pred = pipeline.predict(X_test)

NotFittedError: Vocabulary not fitted or provided

In [68]:
print(classification_report(y_test, y_pred))

NameError: name 'y_pred' is not defined

In [69]:
confusion_matrix(y_test, y_pred)

NameError: name 'y_pred' is not defined

In [ ]:
# das Modell auf ein Beispiel testen

clf.predict(["Das ist eine tolle Uni mit netten Professoren und interessanten Lehrveranstaltungen!"])